In [ ]:
import sys

from torch import nn, optim, save
from torch.utils.data import DataLoader, random_split

from model.dataset import PlantDataset, getTransforms
from model.model import TraitDetector
from model.train import R2_pred, rmse, train, val_eval
from model.utils import set_device

"""Models."""

import logging
import math
from typing import Dict

import torch
from torch import nn
from torchvision.models import ResNet50_Weights, resnet50


class TraitDetector(nn.Module):

    def __init__(self, n_classes, train_features):
        super(TraitDetector, self).__init__()

        # The network is defined as a sequence of operations
        self.resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.resnet.requires_grad_(False)
        self.resnet.fc = nn.Linear(in_features=2048, out_features=train_features)
        # self.resnet.train = lambda x: True
        #
        # self.resnet.training = False

        self.tabular_nn = nn.Sequential(
            nn.Linear(in_features=train_features, out_features=train_features),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=train_features, out_features=train_features),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=train_features, out_features=train_features),
            nn.ReLU(),
        )

        self.merge_nn = nn.Sequential(
            nn.Linear(in_features=2 * train_features, out_features=train_features),
            # nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(in_features=train_features, out_features=n_classes),
        )

    # Specify the computations performed on the data
    def forward(self, x_image, x_row):
        x_image = self.resnet(x_image)
        x_row = self.tabular_nn(x_row)

        return self.merge_nn(torch.cat((x_image, x_row), axis=1))

    def predict(self, x_image, x_row):

        output = self.forward(x_image, x_row)

        return output


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

train_df = pd.read_csv('data/planttraits2024/train.csv')
test_df = pd.read_csv('data/planttraits2024/test.csv')

targets = ["X4_mean", "X11_mean", "X18_mean", "X26_mean", "X50_mean", "X3112_mean"]
sd = ["X4_sd", "X11_sd", "X18_sd", "X26_sd", "X50_sd", "X3112_sd"]

zcored_data = train_df.apply(zscore).drop(axis=1, labels=targets+sd+['id'])
mask = (np.abs(zcored_data) > 3).any(axis=1)

train_df = train_df[~mask]
train_df = train_df.reset_index(drop=True)

sc = StandardScaler()

train_variables = train_df.drop(axis=1, labels=targets+sd+['id'])
column_variables = train_variables.columns

targets_df = train_df[targets+sd+['id']]

test_variables = test_df.drop(axis=1, labels=['id'])
test_ids = test_df[['id']]

transformed_variables = sc.fit_transform(train_variables.values)

transformed_train_df = pd.DataFrame(transformed_variables, columns = column_variables)

transformed_test = sc.transform(test_variables.values)
transformed_test_df = pd.DataFrame(transformed_test, columns=test_variables.columns)

merged_transformed_train_df = targets_df.merge(transformed_train_df, left_index=True, right_index=True)
merged_transformed_test_df = test_ids.merge(transformed_test_df, left_index=True, right_index=True)

merged_transformed_train_df.to_csv('data/planttraits2024/transformed_train_df.csv')
merged_transformed_test_df.to_csv('data/planttraits2024/transformed_test_df.csv')

In [ ]:
import numpy as np
import torch

def R2_pred(y_pred, y_true):
    SS_residuals = torch.pow(y_pred - y_true, 2).sum()
    SS_tot = torch.pow(y_true - y_true.mean(axis=0), 2).sum()
    return 1 - SS_residuals / SS_tot


def rmse(y_pred, y_true):
    return torch.pow(y_pred - y_true, 2).sum()


def train(dataloader, model: TraitDetector, loss_fn, optimizer, scheduler, device):
    model.train()
    train_loss = []

    for i, data in enumerate(dataloader):
        x_image, x_train, y_train = data

        x_image = x_image.to(device, dtype=torch.float)
        x_train = x_train.to(device, dtype=torch.float)
        y_train = y_train.to(device, dtype=torch.float)

        optimizer.zero_grad()

        train_pred = model(x_image, x_train)

        t_loss = loss_fn(train_pred, y_train)

        t_loss.backward()

        optimizer.step()
        train_loss.append(t_loss.cpu().detach().numpy())

    model.eval()
    return np.sqrt(np.sum(train_loss))


def val_eval(dataloader, model, loss_fn, device):
    y_val_list = []
    pred_list = []
    val_loss_list = []
    for i, data in enumerate(dataloader):
        x_img, x_val, y_val = data

        x_img = x_img.to(device, dtype=torch.float)
        x_val = x_val.to(device, dtype=torch.float)
        y_val = y_val.to(device, dtype=torch.float)

        pred = model(x_img, x_val)

        val_loss = loss_fn(pred, y_val)

        y_val_np = y_val.cpu().detach().numpy().tolist()
        pred_np = pred.cpu().detach().numpy().tolist()

        val_loss = val_loss.cpu().detach().numpy()

        val_loss_list.append(val_loss)
        y_val_list.extend(y_val_np)
        pred_list.extend(pred_np)
    #         print(val_loss_list)
    return (
        torch.FloatTensor(y_val_list),
        torch.FloatTensor(pred_list),
        np.sqrt(np.sum(val_loss_list)),
    )


In [ ]:
import pathlib

import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

SIZE = 512


class PlantDataset(Dataset):

    targets = ["X4_mean", "X11_mean", "X18_mean", "X26_mean", "X50_mean", "X3112_mean"]
    sd = ["X4_sd", "X11_sd", "X18_sd", "X26_sd", "X50_sd", "X3112_sd"]

    def __init__(
        self,
        path_to_csv,
        path_to_imgs,
        applied_transforms=None,
        labeled=False,
        num_plants=None,
    ):

        self.path = pathlib.Path(path_to_imgs)

        self.df = pd.read_csv(path_to_csv, dtype={"id": str})

        self.df.set_index(keys=["id"], drop=True, inplace=True)

        if num_plants is not None:
            self.df = self.df.iloc[:num_plants]

        self.train_columns = self.df.columns[
            (~self.df.columns.isin(self.targets)) & (~self.df.columns.isin(self.sd))
        ]

        if applied_transforms:
            self.image_transforms = applied_transforms
        else:
            self.image_transforms = transforms.Compose([transforms.ToTensor()])

        self.labeled = labeled

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):

        plant_id = self.df.index[idx]

        image = Image.open(self.path / f"{plant_id}.jpeg")
        if self.image_transforms:
            image = self.image_transforms(image)

        if self.labeled:
            return (
                image,
                torch.from_numpy(self.df.loc[plant_id, self.train_columns].values),
                torch.from_numpy(self.df.loc[plant_id, self.targets].values),
            )

        return (
            image,
            torch.from_numpy(self.df.loc[plant_id, self.train_columns].values),
            None,
        )


def getTransforms():

    first_transform = [transforms.ToTensor()]

    aug_transforms = [
        transforms.RandomResizedCrop(size=SIZE),
        transforms.RandomRotation(degrees=180),
    ]

    preprocessing_transforms = [  # T.ToTensor(),
        transforms.Resize(size=SIZE),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
    train_transformer = transforms.Compose(
        first_transform + aug_transforms + preprocessing_transforms
    )
    val_transformer = transforms.Compose(first_transform + preprocessing_transforms)
    return train_transformer, val_transformer


In [ ]:

def set_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device != "cuda":
        print("GPU is not enabled in this notebook.")
    else:
        print("GPU is enabled in this notebook. \n")

    return device


DEVICE = set_device()
print(DEVICE)


In [ ]:
r2_est = []
val_loss = []
train_loss = []
N_EPOCHS = 1000
BATCH_SIZE = 100

train_tf, val_tf = getTransforms()

dataset = PlantDataset(
    "data/planttraits2024/transformed_train_df.csv",
    "data/planttraits2024/train_images",
    applied_transforms=train_tf,
    labeled=True,
    # num_plants=2000
)

train_dataset, val_dataset = random_split(dataset, [0.75, 0.25])
# val_dataset = PlantDataset("data/planttraits2024/test.csv", "data/planttraits2024/test_images", applied_transforms=val_tf,
#                            labeled=True)

detector = TraitDetector(n_classes=6, train_features=dataset.train_columns.shape[0])

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(detector.parameters(), lr=0.001, weight_decay=1e-3)
# optimizer = optim.SGD(detector.parameters(), lr=0.001, momentum=.5, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=10, threshold_mode="rel"
)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)

device = DEVICE  # our device (single GPU core)
model = detector.to(device)  # put model onto the GPU core

for epoch in range(N_EPOCHS):
    t_loss = train(train_loader, model, loss_fn, optimizer, scheduler, device)
    y_true, y_pred, v_loss = val_eval(val_loader, model, loss_fn, device)

    r2 = R2_pred(y_pred, y_true)

    logger.info(
        f"R2: {r2:2.3f}, train loss: {t_loss:6,.2f}, val_loss: {v_loss:6,.2f}, learning_rate: {optimizer.param_groups[0]['lr']:1.6f}"
    )

    r2_est.append(r2)

    val_loss.append(v_loss)
    train_loss.append(t_loss)
    scheduler.step(v_loss)

save(model.state_dict(), PATH)

# Load:
#
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()
#